In [1]:
import sys
import scipy.io as sio
import os
import numpy as np
# sys.path.append('/home/astar/Dropbox/share(han-shuji)/KDE/bev/')
# sys.path.append('/home/astar/Dropbox/share(han-shuji)/KDE/4_MRI2atlas_AAL_thr_2hao/')


# prepare data into libsvm format

In [2]:

label_path = '/home/astar/Dropbox/share(han-shuji)/KDE/bev/'
labels_mat = sio.loadmat(os.path.join(label_path,'demo.mat'))
print labels_mat.keys()

['demo', '__version__', '__header__', '__globals__']


In [3]:
print labels_mat['demo'].shape
print str(labels_mat['demo'][2,0][0])
print labels_mat['demo'][3,2].shape

(441, 4)
20071016_02
(1, 1)


In [4]:
labels_dict = {}
num_positive = 0
num_negative = 0
for i in range(1,labels_mat['demo'].shape[0]):
    if len(labels_mat['demo'][i,2]) == 0:
        continue
    label_tmp = 0 if int(labels_mat['demo'][i,2][0,0]) == 2 else 1
    if label_tmp == 0:
        num_negative += 1
    else:
        num_positive += 1
    labels_dict[str(labels_mat['demo'][i,0][0])] = label_tmp
print num_negative, num_positive
    

239 179


In [5]:
feature_path = '/home/astar/Dropbox/share(han-shuji)/KDE/4_MRI2atlas_AAL_thr_2hao/'
features = []
labels = []
for key in labels_dict:
    if os.path.isfile(os.path.join(feature_path, key+'.mat')):
        # Load feature mat file
        key_mat = sio.loadmat(os.path.join(feature_path, key+'.mat'))
        features.append(np.triu(key_mat['Wauc']).flatten().tolist())
        labels.append(labels_dict[key])
    else:
        print key, ' is not in ', feature_path
print len(features), len(labels)
        

20080908_04  is not in  /home/astar/Dropbox/share(han-shuji)/KDE/4_MRI2atlas_AAL_thr_2hao/
20081113_14  is not in  /home/astar/Dropbox/share(han-shuji)/KDE/4_MRI2atlas_AAL_thr_2hao/
20081128_11  is not in  /home/astar/Dropbox/share(han-shuji)/KDE/4_MRI2atlas_AAL_thr_2hao/
20080917_02  is not in  /home/astar/Dropbox/share(han-shuji)/KDE/4_MRI2atlas_AAL_thr_2hao/
20081119_12  is not in  /home/astar/Dropbox/share(han-shuji)/KDE/4_MRI2atlas_AAL_thr_2hao/
20081107_08  is not in  /home/astar/Dropbox/share(han-shuji)/KDE/4_MRI2atlas_AAL_thr_2hao/
20081210_13  is not in  /home/astar/Dropbox/share(han-shuji)/KDE/4_MRI2atlas_AAL_thr_2hao/
20081203_11  is not in  /home/astar/Dropbox/share(han-shuji)/KDE/4_MRI2atlas_AAL_thr_2hao/
20081118_04  is not in  /home/astar/Dropbox/share(han-shuji)/KDE/4_MRI2atlas_AAL_thr_2hao/
20081110_09  is not in  /home/astar/Dropbox/share(han-shuji)/KDE/4_MRI2atlas_AAL_thr_2hao/
20081114_08  is not in  /home/astar/Dropbox/share(han-shuji)/KDE/4_MRI2atlas_AAL_thr_2hao/

In [6]:
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [7]:
names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "AdaBoost",
         "Naive Bayes", "QDA", "Neural Net"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0), warm_start=True),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

In [8]:
# iterate over classifiers
for name, clf in zip(names, classifiers):
    scores = cross_val_score(clf, features, labels, cv=5)
    print name, np.mean(scores)

Nearest Neighbors 0.532316768245
Linear SVM 0.582103844351
RBF SVM 0.606950304735
Gaussian Process 0.582103844351
Decision Tree 0.58494569464
Random Forest 0.589481168933


/usr/local/lib/python2.7/dist-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


AdaBoost 0.661426394749
Naive Bayes 0.684366307236
QDA 0.604136974527


/usr/local/lib/python2.7/dist-packages/sklearn/discriminant_analysis.py:695: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Neural Net 0.532160884513
